In [ ]:
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.path import Path

import scipy.ndimage as nd
import scipy.spatial
from scipy.optimize import minimize

from stsci.convolve import convolve2d
from skimage.transform import warp
from skimage.transform import SimilarityTransform
from skimage.morphology import dilation
from skimage.morphology import binary_dilation

import astropy.io.fits as pyfits
import astropy.wcs as pywcs
import astropy.units as u

from photutils import create_matching_kernel
from photutils import (HanningWindow, TukeyWindow, CosineBellWindow,
                       SplitCosineBellWindow, TopHatWindow)

import drizzlepac
from drizzlepac.astrodrizzle import ablot
import grizli.utils
from golfir import irac 
from golfir.utils import get_wcslist, _obj_shift
from golfir import model
import golfir.model

In [ ]:
root = 'j001420m3024'

PATH = '/Users/vasily/documents/PhD/Projects/IRAC/HST/CHArGE/IRAC/Photometry/HFF-buffalo/'+root
PATH_r=PATH[:-len(root)]
try:
    os.mkdir(PATH)
except:
    pass

os.chdir(PATH)

if not os.path.exists(f'{root}_irac_phot.fits'):
    os.system(f'cp {root}_phot.fits {root}_irac_phot.fits')

In [ ]:
# Download files 
for ext in ['_phot.fits', '-f160w_drz_sci.fits.gz', '-f160w_drz_wht.fits.gz', 
            '-f160w_psf.fits', '-ir_drz_sci.fits.gz', '-ir_drz_wht.fits.gz', '-ir_seg.fits.gz']:
    file=root+ext
    if len(glob.glob(file.strip('.gz')+'*')) == 0:
        print(file)
        os.system(f'wget -O {file} https://s3.amazonaws.com/grizli-v1/Pipeline/{root}/Prep/{file}')

# Gunzip
os.system('gunzip *fits.gz')

# Copy of the catalog for IRAC photometry
if not os.path.exists(f'{root}_irac_phot.fits'):
    os.system(f'cp {root}_phot.fits {root}_irac_phot.fits')

In [ ]:
#Compute photometry

kwargs = {'ds9': None, 
          'patch_arcmin': 1,      # Size of patch to fit
          'patch_overlap': 0.2,     # Overlap of automatic patches
          'mag_limit': [24, 27],    # Two-pass modeling.  Fit sources brighter than mag_limit in HST catalog
          'run_alignment': True,    # Run fine alignment between IRAC-HST, in between two steps of `mag_limit`
          'galfit_flux_limit': 20,  # Brightness limit (uJy) of objects to fit with GALFIT
          'refine_brightest': True, # Refine masked bright objects with galfit
          'any_limit': 15,          # Brightness limit below which to mask *any* sources
          'point_limit': 15,        # Brightness limit below which to mask point-like sources
          'bright_sn': 30,          # S/N threshold for masked pixels of bright object
          'bkg_kwargs': {'order_npix': 64},          # Arguments to the local background routine
          'channels': ['ch1', 'ch2'],  # Channels to try
          'psf_only': False,        
          'use_saved_components': False, # Use models from a "components" file if found
          'window': None            # PSF-match windowing
          }
model.run_all_patches(root,PATH=PATH_r,sync_results=False,**kwargs)

In [ ]:
#Compute photoz

from grizli.pipeline import photoz
import numpy as np

self, cat, zout = photoz.eazy_photoz(root+'_irac', object_only=False, apply_prior=False, 
                                     beta_prior=True, aper_ix=1, force=True, get_external_photometry=False, 
                                     compute_residuals=False, total_flux='flux_auto',
                                    aperture_indices=[1,2])